### Data Science Game 2017
#### Original Features

Data fields

    media_id - identifiant of the song listened by the user
    album_id - identifiant of the album of the song
    media_duration - duration of the song
    user_gender -  gender of the user
    user_id -  anonymized id of the user
    context_type - type of content where the song was listened: playlist, album ...
    release_date - release date of the song with the format YYYYMMDD
    ts_listen - timestamp of the listening in UNIX time
    platform_name - type of os
    platform_family - type of device
    user_age - age of the user
    listen_type - if the songs was listened in a flow or not
    artist_id - identifiant of the artist of the song
    genre_id - identifiant of the genre of the song
    is_listened - 1 if the track was listened, 0 otherwise



### Day count

In [26]:
import pandas as pd

In [27]:
df_train = pd.read_csv("../data/train.csv", nrows=1000)
df_test = pd.read_csv("../data/test.csv")

In [28]:
import datetime
df_train["ts_listen"] = df_train["ts_listen"].apply(lambda x : datetime.datetime.fromtimestamp(x)) 
df_test["ts_listen"] = df_test["ts_listen"].apply(lambda x : datetime.datetime.fromtimestamp(x)) 

In [29]:
df_train.iloc[0]["ts_listen"] - df_train.iloc[1]["ts_listen"]

Timedelta('0 days 14:34:40')

In [30]:
(df_train.iloc[0]["ts_listen"] - df_train.iloc[1]["ts_listen"]).days

0

In [31]:
df_train.iloc[0]["ts_listen"]

Timestamp('2016-12-01 11:00:15')

In [32]:
df_train.iloc[1]["ts_listen"]

Timestamp('2016-11-30 20:25:35')

In [33]:
from greenpyce.feature_engineering import onehot

In [34]:
from greenpyce.feature_engineering import time_count_between_dates

In [35]:
df_train["release_date"].tail()

995    19851231
996    19851231
997    19851231
998    19851231
999    19851231
Name: release_date, dtype: int64

In [36]:
def parse_date(x):
    text = ""
    year = x / 10000
    month = (x % 10000) / 100
    day = (x % 100)
    text = str(year) + "-" + str(month) + "-" + str(day)
    
    return text

#df_train["text"] = df_train["release_date"].apply(lambda x : parse_date(x))
df_test["text"] = df_test["release_date"].apply(lambda x : parse_date(x))

In [37]:
# df_train["text"].head()

In [38]:
#df_train["year"] = df_train["release_date"].apply(lambda x : x / 10000 )
df_test["year"] = df_test["release_date"].apply(lambda x : x / 10000 )

In [39]:
# df_train["year"].describe()

### Year 3000!
Exclude rows with release date after 2017.

In [40]:
#df_train.drop(df_train[df_train["year"] > 2017].index , inplace=True)

In [41]:
#df_train["text"] = df_train["text"].apply(lambda x : pd.to_datetime(x)) 
df_test["text"] = df_test["text"].apply(lambda x : pd.to_datetime(x)) 

In [42]:
#df_train["text"].head()

In [43]:
#df_train["release_date"] = df_train["text"]
df_test["release_date"] = df_test["text"]

In [44]:
#time_count_between_dates(df_train, "release_date", ["ts_listen"]).head()
time_count_between_dates(df_test, "release_date", ["ts_listen"]).head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,text,year,diff_ts_listen_AND_release_date_D,diff_ts_listen_AND_release_date_M,diff_ts_listen_AND_release_date_Y
0,0,50,2016-11-02 14:32:51,683078,82356,1,2002-10-08,0,0,542,1,0,17698,2076,30,2002-10-08,2002,5139.0,171.300000,14.079452
1,1,2744,2016-11-16 15:25:40,876497,99692,1,1985-12-31,0,0,307,1,0,10525,26,28,1985-12-31,1985,11278.0,375.933333,30.898630
2,2,2744,2016-11-19 07:06:01,876497,99692,1,1985-12-31,0,0,307,1,0,8716,26,27,1985-12-31,1985,11281.0,376.033333,30.906849
3,3,2744,2016-11-06 16:42:09,876500,99692,1,1985-12-31,2,1,265,1,0,5443,26,30,1985-12-31,1985,11268.0,375.600000,30.871233
4,4,2744,2016-11-29 17:42:40,876504,99692,1,1985-12-31,2,1,356,1,0,7600,26,29,1985-12-31,1985,11291.0,376.366667,30.934247


In [45]:
#df_train.to_csv("train_data1.csv")

### Round media duration
Seconds are just noise!

In [46]:
df_train["media_duration"].head()

0    223
1    171
2    149
3    240
4    150
Name: media_duration, dtype: int64

In [47]:
#df_train["media_duration"] = df_train["media_duration"].apply(lambda x : round(x / 60.0))
df_test["media_duration"] = df_test["media_duration"].apply(lambda x : round(x / 60.0))
df_train["media_duration"].head()

0    223
1    171
2    149
3    240
4    150
Name: media_duration, dtype: int64

In [48]:
def expand_date_info(df, columns, hour_feats=False):
    """
    Create year, month, day, hour and minute from a datetime column.
    """
    for column in columns:
        df[column + "_DAY"] = df[column].apply(lambda x: x.day)
        df[column + "_MONTH"] = df[column].apply(lambda x: x.month)
        df[column + "_YEAR"] = df[column].apply(lambda x: x.year)
        if hour_feats:
            df[column + "_HOUR"] = df[column].apply(lambda x: x.hour)

    return df

In [49]:
#expand_date_info(df_train, ["release_date"]).head()
expand_date_info(df_test, ["release_date"]).head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,...,artist_id,user_age,text,year,diff_ts_listen_AND_release_date_D,diff_ts_listen_AND_release_date_M,diff_ts_listen_AND_release_date_Y,release_date_DAY,release_date_MONTH,release_date_YEAR
0,0,50,2016-11-02 14:32:51,683078,82356,1,2002-10-08,0,0,9.0,...,2076,30,2002-10-08,2002,5139.0,171.300000,14.079452,8,10,2002
1,1,2744,2016-11-16 15:25:40,876497,99692,1,1985-12-31,0,0,5.0,...,26,28,1985-12-31,1985,11278.0,375.933333,30.898630,31,12,1985
2,2,2744,2016-11-19 07:06:01,876497,99692,1,1985-12-31,0,0,5.0,...,26,27,1985-12-31,1985,11281.0,376.033333,30.906849,31,12,1985
3,3,2744,2016-11-06 16:42:09,876500,99692,1,1985-12-31,2,1,4.0,...,26,30,1985-12-31,1985,11268.0,375.600000,30.871233,31,12,1985
4,4,2744,2016-11-29 17:42:40,876504,99692,1,1985-12-31,2,1,6.0,...,26,29,1985-12-31,1985,11291.0,376.366667,30.934247,31,12,1985


In [50]:
def day_of_week(df, columns):
    """
    Creates a new column with day of week information for given datetime columns.
    0: Monday
    6: Sunday
    """
    for column in columns:
        df[column + "_DAY_OF_WEEK"] = df[column].apply(lambda x: x.weekday())

    return df

In [51]:
#day_of_week(df_train, ["ts_listen"]).head()
day_of_week(df_test, ["ts_listen"]).head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,...,user_age,text,year,diff_ts_listen_AND_release_date_D,diff_ts_listen_AND_release_date_M,diff_ts_listen_AND_release_date_Y,release_date_DAY,release_date_MONTH,release_date_YEAR,ts_listen_DAY_OF_WEEK
0,0,50,2016-11-02 14:32:51,683078,82356,1,2002-10-08,0,0,9.0,...,30,2002-10-08,2002,5139.0,171.300000,14.079452,8,10,2002,2
1,1,2744,2016-11-16 15:25:40,876497,99692,1,1985-12-31,0,0,5.0,...,28,1985-12-31,1985,11278.0,375.933333,30.898630,31,12,1985,2
2,2,2744,2016-11-19 07:06:01,876497,99692,1,1985-12-31,0,0,5.0,...,27,1985-12-31,1985,11281.0,376.033333,30.906849,31,12,1985,5
3,3,2744,2016-11-06 16:42:09,876500,99692,1,1985-12-31,2,1,4.0,...,30,1985-12-31,1985,11268.0,375.600000,30.871233,31,12,1985,6
4,4,2744,2016-11-29 17:42:40,876504,99692,1,1985-12-31,2,1,6.0,...,29,1985-12-31,1985,11291.0,376.366667,30.934247,31,12,1985,1


In [52]:
def period_of_day(df, columns):
    """
    Creates a new column with period of day information for given datetime columns.
    """
    period_dict = {}
    period_dict["Morning"] = {"interval": range(6, 12), "encode": 0}
    period_dict["Afternoon"] = {"interval": range(12, 18), "encode": 1}
    period_dict["Night"] = {"interval": range(18, 24), "encode": 2}
    period_dict["Late"] = {"interval": range(0, 6), "encode": 3}

    def what_period(x, period_dict):
        for key, value in period_dict.items():
            if x in value["interval"]:
                return value["encode"]
        
    for column in columns:
        df[column + "_PERIOD_OF_DAY"] = df[column].apply(lambda x: what_period(x.hour, period_dict))


    return df

In [53]:
period_dict = {}
period_dict["Morning"] = {"interval": range(6, 12), "encode": 0}
period_dict["Afternoon"] = {"interval": range(12, 17), "encode": 1}
period_dict["Night"] = {"interval": range(18, 23), "encode": 2}
period_dict["Late"] = {"interval": range(0, 5), "encode": 3}

def what_period(x, period_dict):
    for key, value in period_dict.items():
        if x in value["interval"]:
            return value["encode"]

In [54]:
#period_of_day(df_train, ["ts_listen"]).head()
period_of_day(df_test, ["ts_listen"]).head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,...,text,year,diff_ts_listen_AND_release_date_D,diff_ts_listen_AND_release_date_M,diff_ts_listen_AND_release_date_Y,release_date_DAY,release_date_MONTH,release_date_YEAR,ts_listen_DAY_OF_WEEK,ts_listen_PERIOD_OF_DAY
0,0,50,2016-11-02 14:32:51,683078,82356,1,2002-10-08,0,0,9.0,...,2002-10-08,2002,5139.0,171.300000,14.079452,8,10,2002,2,1
1,1,2744,2016-11-16 15:25:40,876497,99692,1,1985-12-31,0,0,5.0,...,1985-12-31,1985,11278.0,375.933333,30.898630,31,12,1985,2,1
2,2,2744,2016-11-19 07:06:01,876497,99692,1,1985-12-31,0,0,5.0,...,1985-12-31,1985,11281.0,376.033333,30.906849,31,12,1985,5,0
3,3,2744,2016-11-06 16:42:09,876500,99692,1,1985-12-31,2,1,4.0,...,1985-12-31,1985,11268.0,375.600000,30.871233,31,12,1985,6,1
4,4,2744,2016-11-29 17:42:40,876504,99692,1,1985-12-31,2,1,6.0,...,1985-12-31,1985,11291.0,376.366667,30.934247,31,12,1985,1,1


In [55]:
#df_train.to_csv("train_data1.csv")

### Checkpoint!

In [56]:
import pandas as pd
#df_train = pd.read_csv("train_data1.csv")

### Bin user ages

In [57]:
#df_train["user_age_group"] = pd.cut(df_train["user_age"], [18, 21, 25, 30], labels=[0, 1, 2])
df_test["user_age_group"] = pd.cut(df_test["user_age"], [18, 21, 25, 30], labels=[0, 1, 2])

In [58]:
# df_train["user_age_group"].head()

In [59]:
#df_train["diff_user_song_age"] = df_train["user_age"] - df_train["diff_ts_listen_AND_release_date_Y"]
df_test["diff_user_song_age"] = df_test["user_age"] - df_test["diff_ts_listen_AND_release_date_Y"]

In [60]:
# df_train["release_date_YEAR"].describe()

In [61]:
#df_train["media_decade"] = pd.cut(df_train["release_date_YEAR"], [1900] + [years for years in range(1940, 2030, 10)], labels=[i for i in range(9)])
df_test["media_decade"] = pd.cut(df_test["release_date_YEAR"], [1900] + [years for years in range(1940, 2030, 10)], labels=[i for i in range(9)])

In [62]:
# df_train["media_decade"].head()

In [63]:
# df_train.to_csv("train_data2.csv")
df_test.to_csv("test_data2.csv")